# 3 05 24
I had to restart documentation thanks to a mistake in term 1 that I didn't realise until too late.

Here is my starting code that was made just to make sure I understood the basics of EV3

In [1]:
#!/usr/bin/env pybricks-micropython

from pybricks.hubs import EV3Brick
from pybricks.ev3devices import Motor, UltrasonicSensor, ColorSensor
from pybricks.parameters import Port, Color
from pybricks.tools import wait
from pybricks.robotics import DriveBase
from sympy import Symbol, cos, sin, E

ev3 = EV3Brick()

ev3.speaker.beep()

left_motor = Motor(Port.B)
ev3.speaker.beep()
right_motor = Motor(Port.C)
ev3.speaker.beep()
sensor = UltrasonicSensor(Port.S1)
ev3.speaker.beep()
color_sensor = ColorSensor(Port.S3)
final_diameter = 55.5
final_axle = 104

robot = DriveBase(left_motor, right_motor, final_diameter, final_axle)

robotx = 0
roboty = 0
direction = 0

def Location(x, y):
    x1 = robotx - x
    y1 = roboty - y
    t = Symbol('Θ')
    xr = x1 * cos(t) - (y1 * sin(t))
    yr = x1 * sin(t) - (y1 * cos(t))

    robot.turn()

while True:
    if sensor.distance() > 300:
        robot.drive(200, 0)
        robotx += 200
    else:
        robot.drive(-200, 0)
        robotx -= 200
        robot.turn(90)
        direction += 90
        robot.drive(200, 0)
        roboty += 200
        robot.turn(-90)
        direction -= 90
    if color_sensor.color() == Color.YELLOW:
        ev3.speaker.beep()
    elif color_sensor.color() == Color.RED:
        ev3.speaker.beep()


robotics = DriveBase(left_motor, right_motor, final_diameter, final_axle)

block = False

TypeError: '>' not supported between instances of 'NoneType' and 'int'

## Planning

After planning I started to see how I could code a proper system without relying on the gyro (As Mr Scott and Will both proved it was inaccurate and inconsistent)

### Potential systems
Dynamic locations, [Problem with colour sensor]

Coordinate System, [Could use gyro than return to start if it runs into an error?]

These were the two systems I thought of but I decideds that a coordinate system using trigonometry would be better.

### Coordinate System
Get "North" direction (direction robot starts facing), set as 0,0 with boundaries of at the edges of the box.
Have a subroutine (function) that turns the robot and puts them into a spot.

####    To location function,
Get active coordinate (which will be saved to a variable but it will just be the mm by mm away from 0 it is)
Use trig to get the turn between active coordinate and direction then towards the point.
Then add the turn to the current direction.



In [ ]:
#!/usr/bin/env pybricks-micropython

from pybricks.hubs import EV3Brick
from pybricks.ev3devices import Motor, UltrasonicSensor, ColorSensor
from pybricks.parameters import Port, Color
from pybricks.tools import wait
from pybricks.robotics import DriveBase
import math

ev3 = EV3Brick()

ev3.speaker.beep()

left_motor = Motor(Port.B)
ev3.speaker.beep()
right_motor = Motor(Port.C)
ev3.speaker.beep()
ev3.speaker.beep()
final_diameter = 55.5
final_axle = 104

robot = DriveBase(left_motor, right_motor, final_diameter, final_axle)

robotx = 0
roboty = 0
direction = 0

def Location(x, y):
    global robotx
    global roboty
    global direction
    if (x - y != 0):
        alpha = math.atan((robotx - roboty) / (x - y))
        turndir = math.degrees(alpha)
    else:
        turndir = 0
    direction += turndir
    robot.turn(turndir)
    distance = pow(pow(robotx - x, 2) + pow(roboty - y, 2), .5)
    robotx += x
    roboty += y
    robot.straight(distance) 

Location(500, 500)

# First Attempt
That was my first attempt at the code but for some reason the turning wasn't happening at all, It would travel the correct distance though so atleast I'd made some progress. Over the weekend I went back a reviewed the mathematics.

In [ ]:
#!/usr/bin/env pybricks-micropython
from pybricks.hubs import EV3Brick
from pybricks.ev3devices import Motor
from pybricks.parameters import Port
from pybricks.robotics import DriveBase
import math

robot_x = 0
robot_y = 0
direction = 0

# Initialization
ev3 = EV3Brick()
left_motor = Motor(Port.B)  # Change this to match.
right_motor = Motor(Port.C)  # Change this to match.
robot = DriveBase(left_motor, right_motor, wheel_diameter=55.5, axle_track=104)


def location(x, y):
    global robot_x
    global robot_y
    global direction
    # Calculating Turn
    atan_x = x - robot_x
    atan_y = y - robot_y
    turn = math.atan2(atan_y, atan_x)
    degrees = math.degrees(turn)
    direction = degrees
    robot.turn(degrees)
    print(degrees)  # Debugs how much it should turn by.
    print(direction)  # Debugs where it should be facing.
    distance = pow(pow(x - robot_x, 2) + pow(y - robot_y, 2), 0.5)
    print(distance)  # Debugs distance drove
    robot.straight(distance)
    robot_x = x
    robot_y = y

location(100, 100)
location(0,0)
location(100, 100)

# Something went wrong.
Something about my program wasn't working so I pivoted into making sure the math would work.
(The robot was going to the wrong points)

In [7]:
import math

robot_x = 0
robot_y = 0
direction = 0

def location(x, y):
    global robot_x
    global robot_y
    global direction
    # Calculating Turn
    atan_x = x - robot_x
    atan_y = y - robot_y
    turn = math.atan2(atan_y, atan_x)
    degrees = math.degrees(turn)
    direction = degrees
    print(degrees)  # Debugs how much it should turn by.
    print(direction)  # Debugs where it should be facing.
    distance = pow(pow(x - robot_x, 2) + pow(y - robot_y, 2), 0.5)
    print(distance)  # Debugs distance drove
    robot_x = x
    robot_y = y

location(100, 0)
location(100, 100)
location(0, 100)
location(0, 0)

0.0
0.0
100.0
90.0
90.0
100.0
180.0
180.0
100.0
-90.0
-90.0
100.0


# Output
With this output we can find the error.
Lets break it down:

- First it says it should be facing 0 degrees (Correct), and is turning 0 degrees (Correct). And the distance between (0, 0) and (100, 0) is 100 (Correct).
- Second it says it should be facing 90 degrees (Correct), and is turning 90 degrees (Correct), And the distance between (100, 0) and (100, 100) is 100 (Correct).
- Third it says it should be facing 180 degrees (Correct), and is turning 180 degrees (Wrong), And the distance between (100, 100) and (0, 100) is 100 (Correct).
- Fourth it says it should be facing -90 degrees (Correct), and is turning -90 degrees (Wrong), And the distance between (0, 100) and (0, 0) is 100 (Correct).

As we can see the third line has the error.
I think this is most likely because of some failure of the deciding the difference between facing and turning.
It should be facing 180 degrees (backwards) but it decides to turn 180 degrees instead of turning only 90.
The fourth line also has the error with it turning -90 degrees instead of 90 degrees. Which could be a carried error but also could be the same error as before happening again.

In [8]:
import math

robot_x = 0
robot_y = 0
direction = 0

def location(x, y):
    global robot_x
    global robot_y
    global direction
    # Calculating Turn
    atan_x = x - robot_x
    atan_y = y - robot_y
    turn = math.atan2(atan_y, atan_x)
    degrees = math.degrees(turn)
    tempdirection = degrees
    degrees -= direction
    direction = abs(tempdirection)
    degrees = abs(degrees)
    print(degrees)  # Debugs how much it should turn by.
    print(direction)  # Debugs where it should be facing.
    distance = pow(pow(x - robot_x, 2) + pow(y - robot_y, 2), 0.5)
    print(distance)  # Debugs distance drove
    robot_x = x
    robot_y = y

location(100, 0)
location(100, 100)
location(0, 100)
location(0, 0)

0.0
0.0
100.0
90.0
90.0
100.0
90.0
180.0
100.0
270.0
90.0
100.0


## Pseudo Code for this Subroutine
```
BEGIN subroutine location
INPUT robot x coordinate
INPUT robot y coordinate
INPUT robot direction
INPUT target x coordinate
INPUT target y coordinate

atan x = target x - robot x
atan y = target y - robot y
turn radian = get the atan of atan x and atan y
turn degrees = get turn radian as degrees
turn amount = absolute value of turn degrees - robot direction
robot direction = degrees
distance = square root of (target x - robot x) squared + (target y - robot y) squared.

TURN robot for turn amount
DRIVE robot for distance

OUTPUT robot x = target x
OUTPUT robot y = target y

After Implementing this subroutine into the main code we got no errors with turning.
Meaning the math fix was successful and now the robot coordinate system is functional.